# **LOADING AGGREGATED DATA**

In [1]:
!pip install pandas mysql-connector-python

In [2]:
import os
import json
import pandas as pd
import mysql.connector

In [3]:
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='1234', 
    database='phonepe'
)
cursor = conn.cursor()


In [4]:
base_path = r'C:\Users\sharan\Documents\phonepe_transaction_insights\data\pulse-master\pulse-master\data'

In [5]:
def load_aggregated_user():
    data = []
    path = os.path.join(base_path, 'aggregated', 'user', 'country', 'india', 'state')
    
    for state in os.listdir(path):
        state_path = os.path.join(path, state)
        for year in os.listdir(state_path):
            year_path = os.path.join(state_path, year)
            for file in os.listdir(year_path):
                file_path = os.path.join(year_path, file)
                
                # Skip non-json files
                if not file.endswith('.json'):
                    continue
                
                try:
                    quarter = int(file.strip('.json').split('-')[-1])
                except (IndexError, ValueError):
                    print(f"Skipping file (invalid name): {file}")
                    continue

                with open(file_path, 'r') as f:
                    js = json.load(f)
                    if js['data'] and js['data'].get('usersByDevice'):
                        for u in js['data']['usersByDevice']:
                            data.append({
                                'year': int(year),
                                'quarter': quarter,
                                'state': state.replace('-', ' ').title(),
                                'brand': u.get('brand'),
                                'user_count': u.get('count'),
                                'user_percentage': u.get('percentage')
                            })
    
    df = pd.DataFrame(data)

    for _, row in df.iterrows():
        cursor.execute("""
            INSERT INTO aggregated_user (year, quarter, state, brand, user_count, user_percentage)
            VALUES (%s, %s, %s, %s, %s, %s)
        """, tuple(row))
    conn.commit()
    print("aggregated_user inserted.")


In [6]:
load_aggregated_user()

aggregated_user inserted.


In [7]:
def load_aggregated_transaction():
    base_path = r'C:\Users\sharan\Documents\phonepe_transaction_insights\data\pulse-master\pulse-master\data'
    data = []

    # Walk through the folder structure for transaction data
    for state in os.listdir(os.path.join(base_path, 'aggregated', 'transaction', 'country', 'india', 'state')):
        state_path = os.path.join(base_path, 'aggregated', 'transaction', 'country', 'india', 'state', state)
        if not os.path.isdir(state_path):
            continue

        for year in os.listdir(state_path):
            year_path = os.path.join(state_path, year)
            if not os.path.isdir(year_path):
                continue

            for file_name in os.listdir(year_path):
                if not file_name.endswith('.json'):
                    continue

                quarter = int(file_name.strip('.json').split('-')[-1])
                file_path = os.path.join(year_path, file_name)

                with open(file_path, 'r') as f:
                    content = json.load(f)
                    for tx in content.get('data', {}).get('transactionData', []):
                        data.append({
                            'year': int(year),
                            'quarter': quarter,
                            'state': state.replace('-', ' ').title(),
                            'transaction_type': tx.get('name'),
                            'transaction_count': tx['paymentInstruments'][0].get('count'),
                            'transaction_amount': tx['paymentInstruments'][0].get('amount')
                        })

    df = pd.DataFrame(data)

    # Insert into MySQL table
    for _, row in df.iterrows():
        cursor.execute("""
            INSERT INTO aggregated_transaction 
            (year, quarter, state, transaction_type, transaction_count, transaction_amount)
            VALUES (%s, %s, %s, %s, %s, %s)
        """, tuple(row))

    conn.commit()
    print("aggregated_transaction inserted.")


In [8]:
load_aggregated_transaction()

aggregated_transaction inserted.


In [9]:
import os
import json

def load_aggregated_insurance(base_path, cursor, conn):
    for state in os.listdir(base_path):
        state_path = os.path.join(base_path, state)
        if not os.path.isdir(state_path):
            continue
        for year in os.listdir(state_path):
            year_path = os.path.join(state_path, year)
            if not os.path.isdir(year_path):
                continue
            for file_name in os.listdir(year_path):
                if not file_name.endswith('.json'):
                    continue
                
                quarter = int(file_name.strip('.json').split('-')[-1])  # quarter extraction
                
                file_path = os.path.join(year_path, file_name)
                with open(file_path, 'r') as f:
                    data = json.load(f)
                
                transaction_data = data.get('data', {}).get('transactionData', [])
                for entry in transaction_data:
                    if entry.get('name') == 'Insurance':
                        for payment in entry.get('paymentInstruments', []):
                            insurance_type = payment.get('type', 'Unknown')
                            insurance_count = int(payment.get('count', 0))
                            insurance_amount = float(payment.get('amount', 0))
                            
                            print(f"Inserting: year={year}, quarter={quarter}, state={state}, insurance_type={insurance_type}, count={insurance_count}, amount={insurance_amount}")
                            
                            cursor.execute("""
                                INSERT INTO aggregated_insurance
                                (year, quarter, state, insurance_type, insurance_count, insurance_amount)
                                VALUES (%s, %s, %s, %s, %s, %s)
                            """, (int(year), quarter, state, insurance_type, insurance_count, insurance_amount))
    
    conn.commit()
    print("aggregated_insurance inserted.")


In [10]:
base_path_insurance = r"C:\Users\sharan\Documents\phonepe_transaction_insights\data\pulse-master\pulse-master\data\aggregated\insurance\country\india\state"
load_aggregated_insurance(base_path_insurance, cursor, conn)


Inserting: year=2020, quarter=2, state=andaman-&-nicobar-islands, insurance_type=TOTAL, count=6, amount=1360.0
Inserting: year=2020, quarter=3, state=andaman-&-nicobar-islands, insurance_type=TOTAL, count=41, amount=15380.0
Inserting: year=2020, quarter=4, state=andaman-&-nicobar-islands, insurance_type=TOTAL, count=124, amount=157975.0
Inserting: year=2021, quarter=1, state=andaman-&-nicobar-islands, insurance_type=TOTAL, count=225, amount=244266.0
Inserting: year=2021, quarter=2, state=andaman-&-nicobar-islands, insurance_type=TOTAL, count=137, amount=181504.0
Inserting: year=2021, quarter=3, state=andaman-&-nicobar-islands, insurance_type=TOTAL, count=230, amount=305235.0
Inserting: year=2021, quarter=4, state=andaman-&-nicobar-islands, insurance_type=TOTAL, count=757, amount=1056380.0
Inserting: year=2022, quarter=1, state=andaman-&-nicobar-islands, insurance_type=TOTAL, count=768, amount=1131287.0
Inserting: year=2022, quarter=2, state=andaman-&-nicobar-islands, insurance_type=TOT